In [1]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [2]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from LSTM.ConvLSTM_2 import *
from LSTM.prepare_data_LSTM import *
from LSTM.opt_and_loss import *
from tqdm import tqdm
from time import sleep
import math
from LSTM.torch_early_stopping import  EarlyStopping
import json 
import random

In [3]:
device = 'cpu'
if torch.cuda.is_available():
	torch.set_default_tensor_type(torch.cuda.FloatTensor)
	device = 'cuda'
	print('cuda avaible')

cuda avaible


In [4]:
npy_path = '/content/drive/MyDrive/Colab Notebooks/datasets/qlstm_def'
train_data = ( np.load(npy_path+'/enc_in_train.npy'),
              np.load(npy_path+'/dec_in_train.npy'),
              np.load(npy_path+'/dec_out_train.npy') )

In [5]:

enc_in, dec_in, lab = train_data
enc_in, dec_in, lab = map ( lambda x : torch.from_numpy(x).float().to(device), [enc_in, dec_in, lab] )

In [6]:
n_inputs = enc_in.shape[2] #data_meta_info['n_inputs']
n_outputs = dec_in.shape[2] #data_meta_info['n_outputs']
window_dec = dec_in.shape[1] #data_meta_info['window_dec']


In [7]:
train_set = torch.utils.data.TensorDataset(enc_in, dec_in, lab)
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 16)

In [8]:
hidden_dim=45
kernel_size = (4,4)
model=EncDecConvLSTM( device, n_inputs, hidden_dim, n_outputs, kernel_size) 


model_meta_info = {}
model_meta_info['num layer conv'] = 1
model_meta_info['output_lin_dim'] = [16,1]
model_meta_info['hidden_dim']=hidden_dim
model_meta_info['kernel_size']=kernel_size

In [9]:
n_batch_validation = int(0.2*len(train_loader))
n_batch_train = len(train_loader) - n_batch_validation

In [10]:
n_epochs = 500
opt = torch.optim.Adam(model.parameters(), lr=0.008, betas=(0.9, 0.98), eps=1e-9)
schedule=ScheduledOptim(optimizer=opt,n_warmup_steps=30000,initial_lr=0.008,end_lr=0.0001)
early_stopping=EarlyStopping(patience=15, verbose=True)

In [11]:
loss_fun=torch.nn.MSELoss()

In [12]:
best_model_path = "/content/drive/MyDrive/Colab Notebooks/Thesis utils/Models dict/45_4_DFC_LSTM_for.pt"

In [13]:
model=model.to(device)
best_val_loss=1000.0
lr=[]
train_history=[]
val_history=[]
for epoch in range(n_epochs):
        print(f"learnign rate = {opt.param_groups[0]['lr']}")
        lr.append(opt.param_groups[0]['lr'])
        indexes_of_val_batches = random.sample(range(len(train_loader)),k=n_batch_validation)
        
        val_batches=[]
        train_losses_per_batch=[]
        
        #training
        with tqdm(total=n_batch_train, ncols=100, desc=f'epoch {epoch+1}/{n_epochs}') as bar:
            model.train() 
            for i, batch in enumerate(train_loader):
               
                if i not in indexes_of_val_batches:
                    
                    enc_in, dec_in, lab = batch
                    enc_in=enc_in.to(device)
                    dec_in = dec_in.to(device)
                    lab=lab.to(device)
                    pred = model([enc_in, dec_in])
                    loss = loss_fn(lab, pred)
                    
                    train_losses_per_batch.append(loss.item())
                    mean_loss = np.array(train_losses_per_batch).mean()
                    
                    loss.backward()
                    schedule.step_and_update_lr()
                    schedule.zero_grad()
                    
                    bar.update(1) 
                    bar.set_postfix({'train_loss' : np.round(mean_loss,3)})
                    
                else:
                    val_batches.append(batch)
            model.eval()
            #validation 
            validation_losses_per_batch=[]
            for val_batch in val_batches:
                
                enc_in, dec_in, lab = val_batch
                enc_in=enc_in.to(device)
                dec_in = dec_in.to(device)
                lab=lab.to(device)
                pred = model([enc_in, dec_in])
                val_loss = loss_fn(pred,lab)
                validation_losses_per_batch.append(val_loss.item())
            
            mean_val_loss = np.array(validation_losses_per_batch).mean()
            early_stopping(mean_val_loss, model)
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                torch.save(model.state_dict(), best_model_path)
            if early_stopping.early_stop:
              print("Early stopping")
              break
            bar.set_postfix({'train_loss':np.round(mean_loss,3), 'val_loss':np.round(mean_val_loss,3)}) 
            bar.close() 
        train_history.append(mean_loss.item())
        val_history.append(mean_val_loss.item())

learnign rate = 0.008


epoch 1/500:   0%|                                                          | 0/118 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv2d(input, weight, bias, self.stride,
epoch 1/500: 100%|████████████████| 118/118 [00:13<00:00,  8.94it/s, train_loss=7.89, val_loss=4.01]


learnign rate = 0.007968926666666667


epoch 2/500: 100%|█████████████████| 118/118 [00:06<00:00, 17.56it/s, train_loss=3.2, val_loss=1.81]


learnign rate = 0.007937853333333333


...

learnign rate = 0.0066017


epoch 46/500: 100%|█████████████| 118/118 [00:08<00:00, 14.64it/s, train_loss=0.896, val_loss=0.921]


learnign rate = 0.006570626666666667


epoch 47/500: 100%|██████████████████████████████| 118/118 [00:06<00:00, 17.85it/s, train_loss=0.89]

Early stopping


In [14]:
train_losses_npy=np.array(train_history)
val_losses_npy=np.array(val_history)
path = "/content/drive/MyDrive/Colab Notebooks/Thesis utils/Train losses/history_45_4_DFC"
np.save(f"{path}/train_losses.npy", train_losses_npy)
np.save(f"{path}/val_loss.npy", val_losses_npy)

In [15]:
save_model_json_info_path = '/content/drive/MyDrive/Colab Notebooks/Thesis utils/Meta_info/model_meta_info_45_4_DFC.json'
with open(save_model_json_info_path, 'w') as fp:
    json.dump(model_meta_info, fp, indent=4)